# Tricky Python
和任何语言一样，Python也有许多技巧和陷阱。

## 五坑
原文 [Python这五个坑，怎么避开？](https://mp.weixin.qq.com/s/eBiLE4F9sR_LW7kxle_tLA)

1. 用圆括号()硬编码tuple的时候要注意加逗号。否则圆括号会被当成优先级界定符号。
显式使用`tuple`关键字就不会有这个问题。当然使用`tuple`关键字在python解释的时候会多几个转换字节码

In [2]:
def demo1():
    n = (1)
    print(type(n))
    tup = (1,)
    print(type(tup))

demo1()

<class 'int'>
<class 'tuple'>


2. 默认参数如果是空的容器，切不可写`=[]`这种默认参数

In [3]:
def demo2a():
    def f(a, b=[]):  # “[]” 并不是每次调用都会创建一个新的[]
        print(f"  b is {b}")
        print(f'  id of b is {id(b)}')
        return b
    
    ret = f(1)
    ret.append(1)
    ret.append(2)
    print(f'id of ret is {id(ret)}')
    f(3)

demo2a()

  b is []
  id of b is 1693945914120
id of ret is 1693945914120
  b is [1, 2]
  id of b is 1693945914120


这种情况的正确做法是初始化为`None`, 然后总是判断并创建新的`[]`

In [4]:
def demo2b():
    def f(a, b=None):
        if b is None:
            b = []
        print(f"  b is {b}")
        print(f'  id of b is {id(b)}')
        return b
    
    ret = f(1)
    ret.append(1)
    ret.append(2)
    print(f'id of ret is {id(ret)}')
    f(3)

demo2b()

  b is []
  id of b is 1693945882824
id of ret is 1693945882824
  b is []
  id of b is 1693946919496


3. 外部变量与局部变量重名

In [5]:
def demo3a():
    i = 1
    def f():
        i +=1   # won't work. This is not C/C++. You get UnboundLocalError
        print(f'i in f is {i}')
    
    def g():
        print(f'i in g is {i}')
    
    f()
    g()

demo3a()

UnboundLocalError: local variable 'i' referenced before assignment

In [6]:
def demo3b():
    i = 1   # 像这里在demo3b内部定义的并不是全局变量，下面不能使用global关键字
    def f():
        nonlocal i  # 不能使用global
        i +=1
        print(f'i in f is {i}')
    
    def g():
        print(f'i in g is {i}')
    
    f()
    g()

demo3b()

i in f is 2
i in g is 2


运行下面的代码块前后最好重启下内核，防止全局变量被污染导致结果有偏差

In [7]:
i = 1   # 不是在函数内定义的，应该使用global关键字
def f():
    global i
    i +=1
    print(f'i in f is {i}')

def g():
    print(f'i in g is {i}')

f()
g()

i in f is 2
i in g is 2


4. 在python中`*`与列表操作，实现快速元素复制。但如果不注意使用条件的话就会犯错。

对于浅层复制，这样做没有问题

In [8]:
def demo4a():
    a = [1,3,5] * 3
    print(f'a is {a}')
    a[0] = 10
    print(f'then a is {a}')

demo4a()

a is [1, 3, 5, 1, 3, 5, 1, 3, 5]
then a is [10, 3, 5, 1, 3, 5, 1, 3, 5]


但是对于嵌套的容器，这么做会发生什么？

In [9]:
def demo4b():
    a = [[1,3,5],[2,4]] * 3
    print(f'a is {a}')
    a[0][0] = 10
    print(f'then a is {a}')
    print('id of a is')
    print([ id(i) for i in [x for x in a] ])

demo4b()

a is [[1, 3, 5], [2, 4], [1, 3, 5], [2, 4], [1, 3, 5], [2, 4]]
then a is [[10, 3, 5], [2, 4], [10, 3, 5], [2, 4], [10, 3, 5], [2, 4]]
id of a is
[1693945869640, 1693946847560, 1693945869640, 1693946847560, 1693945869640, 1693946847560]


正如你所见，这一次的`a[0]`,`a[2]`,`a[4]`的id是一样的（当然`a[1]`,`a[3]`,`a[5]`的也是一样的）。原来`*`进行的是浅拷贝，一变俱变！

正确的做法是显式使用深拷贝。既可以使用`copy.deepcopy`，也可以显式创建新的`list`

In [10]:
def demo4c():
    a = [ [1,3,5] if i%2==0 else [2,4] for i in range(6) ]
    print(a)
    a[0][0] = 10
    print(f'then a is {a}')
    print('id of a is')
    print([ id(i) for i in [x for x in a] ])
    
demo4c()

[[1, 3, 5], [2, 4], [1, 3, 5], [2, 4], [1, 3, 5], [2, 4]]
then a is [[10, 3, 5], [2, 4], [1, 3, 5], [2, 4], [1, 3, 5], [2, 4]]
id of a is
[1693946762248, 1693946849096, 1693947247112, 1693946891400, 1693946765064, 1693945499272]


5. 删除列表中的重复元素，没删干净

In [11]:
def demo5a():
    def del_item(lst,x):
        [lst.remove(i) for i in lst if i==x] # NO!
        return lst
    print(del_item([1,3,3,3,5],3))   # 中间的“3”删不掉

demo5a()

[1, 3, 5]


正确做法是“反其道行之”，也就是不做删除，而是将要删元素之外的元素取出

In [12]:
def demo5b():
    def del_item(lst,x):
        return [i for i in lst if i!=x]
    print(del_item([1,3,3,3,5],3))

demo5b()

[1, 5]
